In [1]:
import DifferentialEquations as DE

In [ ]:
datapath = "D:\\MCFdata\\w7x\\"
println( Threads.nthreads() )
using Jynamics
using TensorCast

In [3]:
rminb = 4.0;
rmaxb = 7.0;
zminb = -1.5;
zmaxb = 1.5;
nSym = 5;

R = collect( range(rminb, stop=rmaxb, length=128) );
Z = collect( range(zminb, stop=zmaxb, length=128) );
Phi = collect( range(0.0, stop=2pi/nSym, length=65) );


In [4]:
BR = Array{Float64,3}(undef, 128, 128, 65);
BZ = Array{Float64,3}(undef, 128, 128, 65);
BPhi = Array{Float64,3}(undef, 128, 128, 65);
BR_pert = Array{Float64,3}(undef, 128, 128, 65);
BZ_pert = Array{Float64,3}(undef, 128, 128, 65);
BPhi_pert = Array{Float64,3}(undef, 128, 128, 65);

using NPZ
Bfield_np = npzread(datapath * "w7x-op21-standard.npy");
Bfield_pert_np = npzread(datapath * "w7x-op21-controlCoils1.npy");

In [5]:
for iPhi = 1:length(Phi)-1
    BR[:,:,iPhi] = Bfield_np[:,:,iPhi,1];
    BZ[:,:,iPhi] = Bfield_np[:,:,iPhi,2];
    BPhi[:,:,iPhi] = Bfield_np[:,:,iPhi,3];
end
BR[:,:,end] = Bfield_np[:,:,1,1];
BZ[:,:,end] = Bfield_np[:,:,1,2];
BPhi[:,:,end] = Bfield_np[:,:,1,3];

B_unperturbed = CylindricalVectorField(R, Z, Phi, BR, BZ, BPhi; nSym=nSym);


In [6]:
for iPhi = 1:length(Phi)-1
    BR_pert[:,:,iPhi] = Bfield_pert_np[:,:,iPhi,1];
    BZ_pert[:,:,iPhi] = Bfield_pert_np[:,:,iPhi,2];
    BPhi_pert[:,:,iPhi] = Bfield_pert_np[:,:,iPhi,3];
end
BR_pert[:,:,end] = Bfield_pert_np[:,:,1,1];
BZ_pert[:,:,end] = Bfield_pert_np[:,:,1,2];
BPhi_pert[:,:,end] = Bfield_pert_np[:,:,1,3];

B_pert = CylindricalVectorField(R, Z, Phi, BR_pert, BZ_pert, BPhi_pert; nSym=5);

In [ ]:
B_total = B_unperturbed + B_pert

In [8]:
import DifferentialEquations as DE

In [9]:
# using GridInterpolations
using Interpolations

In [11]:

BR_interp = linear_interpolation((R,Z,Phi), BR);
BZ_interp = linear_interpolation((R,Z,Phi), BZ);
BPhi_interp = linear_interpolation((R,Z,Phi), BPhi);

In [12]:
using TensorCast
using Memoization

In [13]:
@cast RBRoBPhi[iR,iZ,iPhi] := R[iR] * BR[iR,iZ,iPhi] / BPhi[iR,iZ,iPhi];
@cast RBZoBPhi[iR,iZ,iPhi] := R[iR] * BZ[iR,iZ,iPhi] / BPhi[iR,iZ,iPhi];

In [ ]:
size(RBRoBPhi)

In [ ]:
dR = R[2] - R[1]
dZ = Z[2] - Z[1]

        

In [ ]:

FLT_A =  RVpoloVPhi_pRpZ_interp(B_unperturbed)

In [ ]:
FLT_A(5.0, 0.0, 0.0)

In [18]:

BRoBPhi_delta_DeltaB = BR_pert ./ BPhi .- BR .* BPhi_pert ./ (BPhi.^2);
BRoBPhi_delta_DeltaB_interp = linear_interpolation( (R,Z,Phi), BRoBPhi_delta_DeltaB );
BZoBPhi_delta_DeltaB = BZ_pert ./ BPhi .- BZ .* BPhi_pert ./ (BPhi.^2);
BZoBPhi_delta_DeltaB_interp = linear_interpolation( (R,Z,Phi), BZoBPhi_delta_DeltaB );
@inline RBpoloBPhi_delta_DeltaB(r,z,phi) = r*[BRoBPhi_delta_DeltaB_interp(r,z,phi); BZoBPhi_delta_DeltaB_interp(r,z,phi)];

In [ ]:
function dXpol_dphi!(dx,x,p,phi)
    phimod = mod( phi, 2pi/5 )
    dx[1] = x[1] * BR_interp(x[1], x[2], phimod) / BPhi_interp(x[1], x[2], phimod)
    dx[2] = x[1] * BZ_interp(x[1], x[2], phimod) / BPhi_interp(x[1], x[2], phimod)
end


In [ ]:
condition(x,t,integrator) = x[1]>R[end-3] || x[1] < R[3] || x[2] >Z[end-3] || x[2] < Z[3]
affect!(integrator) = DE.terminate!(integrator)
cb = DE.DiscreteCallback(condition,affect!)

In [22]:
roundN = 30.0
trajN = 16
# x0s = Array{Float64,2}(undef, trajN, 3);
# x0s[:,1] = range(6.0, stop=6.0, length=trajN);
# x0s[:,2].= 0.0;
# x0s[:,3] = range(-0.8, stop=0.8, length=trajN);

x0s = Array{Float64,2}(undef, trajN, 2);
x0s[:,1] = range(6.2, stop=5.4, length=trajN);
x0s[:,2] = range(-0.6, stop=1.0, length=trajN);

tspan = (0.0, roundN*2pi)
# prob = DE.ODEProblem(FLT_cartesian!, x0s[1,:], tspan)
prob = DE.ODEProblem(dXpol_dphi!, x0s[1,:], tspan)

function prob_func(prob, i, repeat)
    DE.remake(prob, u0=x0s[i,:])
end
ensemble_prob = DE.EnsembleProblem(prob, prob_func=prob_func)
# sol = solve(prob, RK4(), abstol=1e-9, reltol=1e-9, maxiters=1e9, dt=pi/128,) #dt=pi/128, dtmax=pi/128
ensemble_sol = DE.solve(
    ensemble_prob, DE.RK4(), DE.EnsembleThreads(), trajectories=trajN, 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128, callback=cb );

In [ ]:
import Plots
scatxy = reduce( hcat, [ensemble_sol[1](2pi*i) for i in range(0, min( maximum(ensemble_sol[1].t)/2pi, roundN) )] )
scatx, scaty = scatxy[1,:], scatxy[2,:]
Plots.scatter(scatx, scaty, aspect_ratio=1.0, size=(600,1000), ms=0.2)
for traj_i in range(2, trajN-1)
    scatxy = reduce( hcat, [ensemble_sol[traj_i](2pi*i) for i in range(0, min( maximum(ensemble_sol[traj_i].t)/2pi, roundN) )] )
    scatx, scaty = scatxy[1,:], scatxy[2,:]
    Plots.scatter!(scatx, scaty, ms=0.2)
end
Plots.scatter!()

In [ ]:

function dDXpol_dphi(dDXpol,DXpol,p,phi)
    phimod = mod( phi, 2pi/5 )
    traj_i = p[1]
    r,z = ensemble_sol[traj_i](phi)
    dDXpol[:,:] = FLT_A(r,z,phimod) * DXpol
end

In [ ]:

prob_DXpol = DE.ODEProblem(dDXpol_dphi, [1 0; 0 1], tspan, [0])
function prob_func_DXpol(prob, i, repeat)
    DE.remake(prob, tspan=[0.0, maximum(ensemble_sol[i].t) ], p=[i] )
end
ensemble_prob_DXpol = DE.EnsembleProblem(prob_DXpol, prob_func=prob_func_DXpol)
# sol = solve(prob, RK4(), abstol=1e-9, reltol=1e-9, maxiters=1e9, dt=pi/128,) #dt=pi/128, dtmax=pi/128
ensemble_sol_DXpol = DE.solve(
    ensemble_prob_DXpol, DE.RK4(), DE.EnsembleThreads(), trajectories=trajN, 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128, )

In [26]:
import DifferentialEquations as DE
import LinearAlgebra as LA


# function dDXpol_dphi!(dDXpol,DXpol,p,phi)
#     phimod = mod( phi, 2pi/5 )
#     sol_Xpol, = p
#     r,z = sol_Xpol(phi)
#     dDXpol[:,:] = FLT_A(r,z,phimod) * DXpol
# end

# function lookfor_cycle(Bfield, init_rzphi, tor_turn=1, h=0.98, delta_threshold=1e-5)
#     phi = init_rzphi[3]
#     tspan = (phi, phi + tor_turn*2pi)
#     x0rz = init_rzphi[1:2]
    
#     while true
#         _prob_Xpol = DE.ODEProblem(dXpol_dphi!, x0rz, tspan)
#         _sol_Xpol = DE.solve(_prob_Xpol, DE.RK4(), 
#             abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128, callback=cb) 
        
#         _prob_DXpol = DE.ODEProblem(dDXpol_dphi!_generator(sols_dict), [1 0; 0 1], tspan)
        
#         _sol_DXpol = DE.solve(_prob_DXpol, DE.RK4(), 
#             abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)
        
#         delta =  - h * LA.inv(_sol_DXpol(phi + tor_turn*2pi) - LA.I(2) ) * (_sol_Xpol(phi + tor_turn*2pi) - x0rz) 
#         x0rz += delta
#         # println(x0rz)
#         if LA.norm(delta) < delta_threshold
#             break
#         end
#     end
#     return push!(x0rz, phi)
# end

In [ ]:

cyc_init_rzphi = lookfor_cycle(B_unperturbed, [6.05; 0.6; 0.0])
# cyc_init_rzphi = lookfor_cycle(0.0, [6.05;-0.6; 0.0])
# cyc_init_rzphi = lookfor_cycle(0.0, [5.5; 0.8; 0.0])
# cyc_init_rzphi = lookfor_cycle(0.0, [5.5;-0.8; 0.0])
# cyc_init_rzphi = lookfor_cycle(0.0, [5.7; 0.0; 0.0])

In [ ]:
phi = cyc_init_rzphi[3]
tor_turn = 1
tspan = (phi, phi + tor_turn*2pi)

sols_dict = get_tracing_sols(B_unperturbed, cyc_init_rzphi[1:2], cyc_init_rzphi[3])

prob_Xpol = DE.ODEProblem(dXpol_dphi!_generator(sols_dict), cyc_init_rzphi[1:2], tspan)
sol_Xpol = DE.solve(prob_Xpol, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128, callback=cb) 
sols_dict["sol_Xpol"] = sol_Xpol

prob_DXpol = DE.ODEProblem(dDXpol_dphi!_generator(sols_dict), [1 0; 0 1], tspan)
# prob_DXpol = DE.ODEProblem(dDXpol_dphi!, [1 0; 0 1], tspan, [sol_Xpol,])
sol_DXpol = DE.solve(prob_DXpol, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)
sols_dict["sol_DXpol"] = sol_DXpol

In [ ]:
# function dDPm_dphi!(dDPm,DPm,p,phi)
#     phimod = mod( phi, 2pi/5 )
#     sol_Xpol, = p
#     r,z = sol_Xpol(phi)
#     dDPm[:,:] = FLT_A(r,z,phimod) * DPm - DPm * FLT_A(r,z,phimod) 
# end



prob_DPm = DE.ODEProblem(dDPm_dphi!_generator(sols_dict), sol_DXpol(tspan[2]), tspan,)
sol_DPm = DE.solve(prob_DPm, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)

In [ ]:
function dXpol_delta_dphi!(dXpol_delta, Xpol_delta, p, phi)
    phimod = mod( phi, 2pi/5 )
    sol_Xpol, = p
    r,z = sol_Xpol(phi)
    dXpol_delta[:] = FLT_A(r,z,phimod) * Xpol_delta + RBpoloBPhi_delta_DeltaB(r,z,phimod) 
end

prob_Xpol_delta = DE.ODEProblem(dXpol_delta_dphi!, [0;0], tspan, [sol_Xpol,])
sol_Xpol_delta = DE.solve(prob_Xpol_delta, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)

In [ ]:
Xcyc_init_delta_Delta = LA.inv( sol_DPm(phi) - LA.I(2) ) * (- sol_Xpol_delta(2pi) )

In [ ]:
dXcyc_delta_dphi! = dXpol_delta_dphi!

prob_Xcyc_delta = DE.ODEProblem(dXcyc_delta_dphi!, Xcyc_init_delta_Delta, tspan, [sol_Xpol,])
sol_Xcyc_delta = DE.solve(prob_Xcyc_delta, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)

In [ ]:
using Plots
plot(sol_Xcyc_delta)

In [ ]:
# RBRoBPhi_pR2 = pRpZ_interp(RBRoBPhi,2,0);
# RBRoBPhi_pRpZ = pRpZ_interp(RBRoBPhi,1,1);
# RBRoBPhi_pZ2 = pRpZ_interp(RBRoBPhi,0,2);
# RBZoBPhi_pR2 = pRpZ_interp(RBZoBPhi,2,0); 
# RBZoBPhi_pRpZ = pRpZ_interp(RBZoBPhi,1,1);
# RBZoBPhi_pZ2 = pRpZ_interp(RBZoBPhi,0,2);
RBRoBPhi_pR2  = linear_interpolation( (R,Z,Phi), 2*pRpZ(BR./BPhi,1,0,R,Z) + R.*pRpZ(BR./BPhi,2,0,R,Z) );
RBRoBPhi_pRpZ = linear_interpolation( (R,Z,Phi),   pRpZ(BR./BPhi,0,1,R,Z) + R.*pRpZ(BR./BPhi,1,1,R,Z) );
RBRoBPhi_pZ2  = linear_interpolation( (R,Z,Phi),                            R.*pRpZ(BR./BPhi,0,2,R,Z) );
RBZoBPhi_pR2  = linear_interpolation( (R,Z,Phi), 2*pRpZ(BZ./BPhi,1,0,R,Z) + R.*pRpZ(BZ./BPhi,2,0,R,Z) );
RBZoBPhi_pRpZ = linear_interpolation( (R,Z,Phi),   pRpZ(BZ./BPhi,0,1,R,Z) + R.*pRpZ(BZ./BPhi,1,1,R,Z) );
RBZoBPhi_pZ2  = linear_interpolation( (R,Z,Phi),                            R.*pRpZ(BZ./BPhi,0,2,R,Z) );
@inline A_dirderivative(r,z,phi,dir_r,dir_z) = [ 
    RBRoBPhi_pR2(r,z,phi) * dir_r + RBRoBPhi_pRpZ(r,z,phi) * dir_z       RBRoBPhi_pRpZ(r,z,phi) * dir_r + RBRoBPhi_pZ2(r,z,phi)  * dir_z; 
    RBZoBPhi_pR2(r,z,phi) * dir_r + RBZoBPhi_pRpZ(r,z,phi) * dir_z       RBZoBPhi_pRpZ(r,z,phi) * dir_r + RBZoBPhi_pZ2(r,z,phi)  * dir_z]

In [ ]:
# plot(sol_Xcyc_delta)
# plot(sol_DXpol)

plot( [prod(LA.eigvals(sol_DPm(phi)) ) for phi in 0.0:2pi/100:2pi] )
# plot( [prod(LA.eigvals(sol_DXpol(phi)) ) for phi in 0.0:2pi/100:2pi] )

In [ ]:
A_delta_DeltaB = RVpoloVPhi_pRpZ_delta_v_pert_interp(B_unperturbed, B_pert)

function dXpol_delta_dphi!(dXpol_delta, Xpol_delta, p, phi)
    phimod = mod( phi, 2pi/5 )
    sol_Xpol, sol_DXpol, sol_Xcyc_delta = p
    r,z = sol_Xpol(phi)
    dr, dz = sol_Xcyc_delta(phi)
    # println( A_delta_DeltaB(r,z,phimod)  )
    # println( A_dirderivative(r,z,phimod,dr,dz) ) 
    dXpol_delta[:,:] = (A_delta_DeltaB(r,z,phimod) + A_dirderivative(r,z,phimod,dr,dz) ) * sol_DXpol(phi) + FLT_A(r,z,phimod) * Xpol_delta
end


prob_Xpol_delta = DE.ODEProblem(dXpol_delta_dphi!, zeros(2,2), tspan, [sol_Xpol, sol_DXpol, sol_Xcyc_delta])
sol_Xpol_delta = DE.solve(prob_Xpol_delta, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)

In [ ]:
using Plots
# plot(LA.eigvals(sol_DPm))
# plot(sol_DXpol)
plot(sol_Xpol_delta)

In [ ]:
function dDPm_delta_dphi!(dDPm_delta, DPm_delta, p, phi)
    phimod = mod( phi, 2pi/5 )
    sol_Xpol, sol_DPm, sol_Xcyc_delta = p
    r,z = sol_Xpol(phi)
    dr, dz = sol_Xcyc_delta(phi)
    dDPm_delta[:,:] = (
        (A_delta_DeltaB(r,z,phimod) + A_dirderivative(r,z,phimod,dr,dz) ) * sol_DPm(phi) 
        - sol_DPm(phi) * (A_delta_DeltaB(r,z,phimod) + A_dirderivative(r,z,phimod,dr,dz) ) 
        + FLT_A(r,z,phimod) * DPm_delta 
        - DPm_delta * FLT_A(r,z,phimod)
    )
    # println(   FLT_A(r,z,phimod) * DPm_delta_DeltaB   )
end

prob_DPm_delta_DeltaB = DE.ODEProblem(dDPm_delta_dphi!, sol_Xpol_delta(tspan[2]), tspan, [sol_Xpol, sol_DPm, sol_Xcyc_delta])
sol_DPm_Delta = DE.solve(prob_DPm_delta_DeltaB, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128/2,);

In [ ]:
plot(sol_DPm_Delta)

In [ ]:
sol_DPm_Delta(0.0)

In [ ]:
sol_DPm_Delta(2pi)

In [ ]:
plot()
for i in 0.0:0.02:1
    plot!([ prod(
            LA.eigvals(sol_DPm(2pi*i) + len * sol_DPm_Delta(2pi*i)  )
        ) for len in -2.5:0.001:2.5] )
end
plot!()

In [ ]:
plot()

plot!(
    [2pi*phifrac for phifrac in 0.0:0.02:1],
    [LA.eigvals(sol_DPm(2pi* phifrac) )[1] for phifrac in 0.0:0.02:1]
)
plot!(
    [2pi*phifrac for phifrac in 0.0:0.02:1],
    [LA.eigvals(sol_DPm(2pi* phifrac) )[2] for phifrac in 0.0:0.02:1]
)   
    
plot!(
    [2pi*phifrac for phifrac in 0.0:0.02:1],
    [LA.eigvals( sol_DPm(2pi* phifrac) + sol_DPm_Delta(2pi*phifrac)  )[1] - LA.eigvals(sol_DPm(2pi* phifrac) )[1] for phifrac in 0.0:0.02:1]
)
plot!(
    [2pi*phifrac for phifrac in 0.0:0.02:1],
    [LA.eigvals( sol_DPm(2pi* phifrac) + sol_DPm_Delta(2pi*phifrac)  )[2] - LA.eigvals(sol_DPm(2pi* phifrac) )[2] for phifrac in 0.0:0.02:1]
)   
    
plot!()

In [ ]:

VectorMatrix2Array(sol_DPm.u)[1,:,:]

In [ ]:
using HDF5
h5open(datapath * "w7x_standard_cycle1.h5", "w") do fid
    g = create_group(fid, "cyc")
    g["t_Xpol"] = sol_Xpol.t
    g[  "Xpol"] = VectorVector2Array(sol_Xpol.u)
    g["t_DXpol"] = sol_DXpol.t
    g[  "DXpol"] = VectorMatrix2Array(sol_DXpol.u)
    g["t_DPm"] = sol_DPm.t
    g[  "DPm"] = VectorMatrix2Array(sol_DPm.u)
    g["t_Xpol_delta"] = sol_Xpol_delta.t
    g[  "Xpol_delta"] = VectorMatrix2Array(sol_Xpol_delta.u)
    g["t_Xcyc_delta"] = sol_Xcyc_delta.t
    g[  "Xcyc_delta"] = VectorVector2Array(sol_Xcyc_delta.u)
    g["t_DPm_Delta"] = sol_DPm_Delta.t
    g[  "DPm_Delta"] = VectorMatrix2Array(sol_DPm_Delta.u)
end;

In [49]:

using HDF5
h5open(datapath * "w7x_standard_phi_0_rad_Poincare.h5", "w") do fid
    for traj_i in range(2, trajN-1)
        scatxy = reduce( hcat, [ensemble_sol[traj_i](2pi*i) for i in range(0, floor(ensemble_sol[traj_i].t[end]/2pi) ) ] )
        fid["traj_$traj_i"] = permutedims(scatxy)
    end;
end;

h5open(datapath * "w7x_standard_phi_2pio20_rad_Poincare.h5", "w") do fid
    for traj_i in range(2, trajN-1)
        scatxy = reduce( hcat, [ensemble_sol[traj_i](2pi/20 + 2pi*i) for i in range(0, floor( (ensemble_sol[traj_i].t[end] - 2pi/20 )/2pi) ) ] )
        fid["traj_$traj_i"] = permutedims(scatxy)
    end;
end;

h5open(datapath * "w7x_standard_phi_2pio10_rad_Poincare.h5", "w") do fid
    for traj_i in range(2, trajN-1)
        scatxy = reduce( hcat, [ensemble_sol[traj_i](2pi/10 + 2pi*i) for i in range(0, floor( (ensemble_sol[traj_i].t[end] - 2pi/10 )/2pi) ) ] )
        fid["traj_$traj_i"] = permutedims(scatxy)
    end;
end;

h5open(datapath * "w7x_standard_phi_6pio20_rad_Poincare.h5", "w") do fid
    for traj_i in range(2, trajN-1)
        scatxy = reduce( hcat, [ensemble_sol[traj_i](6pi/20 + 2pi*i) for i in range(0, floor( (ensemble_sol[traj_i].t[end] - 6pi/20 )/2pi) ) ] )
        fid["traj_$traj_i"] = permutedims(scatxy)
    end;
end;



In [53]:
# NPZ support for multidimensional array lacks
using NPZ

dict_to_store = Dict{String, AbstractArray}();

dict_to_store["t_Xpol"] = sol_Xpol.t;
dict_to_store["Xpol"] = VectorVector2Array(sol_Xpol.u);
# dict_to_store["Xpol_delta"] = VectorVector2Array(sol_Xcyc_delta.u);
npzwrite("w7x_standard_cycle1.npz", dict_to_store);
# "t_DXpol" => sol_DXpol.t, "DXpol" => VectorMatrix2Array(sol_DXpol.u),
# "t_DPm" => sol_DPm.t, "DPm" => VectorMatrix2Array(sol_DPm.u),
# "t_Xpol_delta" => sol_Xcyc_delta.t, "Xpol_delta" => VectorVector2Array(sol_Xcyc_delta.u),
# "t_Xpol_delta" => sol_Xpol_delta.t, "Xpol_delta" => VectorMatrix2Array(sol_Xpol_delta.u),
# "t_DPm_delta_DeltaB" => sol_DPm_delta.t, "DPm_delta_DeltaB" => VectorMatrix2Array(sol_DPm_delta.u)


In [ ]:
import LinearAlgebra as LA
using Plots

for traj_i = 1:1
    DXpol_Phi = ensemble_sol_DXpol[traj_i].t[1:10:end]
    DXpol_eigvals = reduce(vcat, [LA.eigvals(DXpol)' for DXpol in ensemble_sol_DXpol[traj_i].u[1:10:end] ] )
    plot(DXpol_Phi, DXpol_eigvals)
end

In [ ]:
import LinearAlgebra as LA
using Plots
traj_i = 12
roundN_todraw = 500

DXpol_eigvals = reduce(vcat, [LA.eigvals(ensemble_sol_DXpol[traj_i](2pi*i))'  for i in range(0,roundN_todraw)] )
scatter( range(0,roundN_todraw), abs.(DXpol_eigvals[:,1]), labels="abs(λ1)")
scatter!( range(0,roundN_todraw), abs.(DXpol_eigvals[:,2]), labels="abs(λ2)" )

    

In [56]:
using NPZ
orb_dict = Dict{String, AbstractArray}()
for traj_i in range(1, trajN)
#     scatRZPhi = hcat( reduce( hcat, [ensemble_sol[traj_i](2pi*i) for i in range(0, roundN)] )', [2pi*i for i in range(0, roundN) ] )
#     println(size(scatRZPhi))
    orb_dict[string(traj_i)] = hcat( reduce( hcat, [ensemble_sol[traj_i](2pi*i) for i in range(0, roundN)] )', [2pi*i for i in range(0, roundN) ] )
#     Plots.scatter!(scatx, scaty)
end

npzwrite("w7x_standard_phi_0_Poincare_orbits.npz", orb_dict )

In [58]:
using CSV, DataFrames

for traj_i in range(1, trajN)
    orb_filename = datapath * "w7x_standard_" * string(traj_i) * ".csv"
    touch(orb_filename)
    file_handle = open(orb_filename, "w")
    scatRZ = reduce( hcat, [ensemble_sol[traj_i](2pi*i) for i in range(0, roundN)] )
    scatR, scatZ = scatRZ[1,:], scatRZ[2,:]
    CSV.write(orb_filename, DataFrame(R = scatR, Z = scatZ, Phi = [2pi*i for i in range(0, roundN)] ) )
end




In [ ]:
npzwritearray(npzfile_orb,  hcat( reduce( hcat, [ensemble_sol[1](2pi*i) for i in range(0, 20)] )', [2pi*i for i in range(0, 20) ] )  )

In [ ]:
reduce( hcat, [ensemble_sol[1](2pi*i) for i in range(0, roundN)] )